下面是在http://58.241.217.181:15111/notebooks/work/ml/gnt-a/notebooks/code/customized-model-A.ipynb#
的基础上，参照：
https://www.tensorflow.org/versions/master/tutorials/layers
来制作CNN，用于识别GNT.



In [1]:
import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

IMAGE_SIZE = 64
# CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 64, 64, 1])
  print ('shape of input_layer: ', input_layer)
  # with batch_size =100, shape should be: [100, 28, 28, 1]
  # shape of input_layer:  Tensor("Reshape:0", shape=(100, 64, 64, 1), dtype=float32)

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  print ('shape of conv1: ', conv1)
  # shape of conv1:  Tensor("conv2d/Relu:0", shape=(100, 64, 64, 32), dtype=float32)
    
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  print ('shape of conv2: ', conv2, '; and shape of pool2 is: ', pool2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=CHARSET_SIZE)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


class DataSetLoader:
    def __init__(self, data_dir):
        # Set FLAGS.charset_size to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        print ('self.images: ', self.images[0].dtype)
        self.images = np.float32(self.images)
        print ('self.images: ', self.images[0].dtype)

        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

    
train_data = DataSetLoader(data_dir='../data/train_/')
print ('Train data loaded ...')


# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]



# Build CNN with customized function ...
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,
   model_dir='../dfs/checkpoint/customized_model-c')

    

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(train_data)[0]},
    y=input(train_data)[1],
    num_epochs=None,
    batch_size=100,
    shuffle=True
)
print ('Begin to train ...')

classifier.train(input_fn=train_input_fn, steps=100000)
print ('Train done ...')

test_data = DataSetLoader(data_dir='../data/test_/')
print ('Test data loaded ...')

# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

Now processing path:  ../data/train_/00037
self.images:  float64
self.images:  float32
Train data loaded ...
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../dfs/checkpoint/customized_model-c', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
Begin to train ...
shape of input_layer:  Tensor("Reshape:0", shape=(100, 64, 64, 1), dtype=float32)
shape of conv1:  Tensor("conv2d/Relu:0", shape=(100, 64, 64, 32), dtype=float32)
shape of conv2:  Tensor("conv2d_2/Relu:0", shape=(100, 32, 32, 64), dtype=float32) ; and shape of pool2 is:  Tensor("max_pooling2d_2/MaxPool:0", shape=(100, 16, 16, 64), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-145101
INFO:tensorflow:Saving 

INFO:tensorflow:global_step/sec: 1.1461
INFO:tensorflow:loss = 0.00184306, step = 151402 (87.252 sec)
INFO:tensorflow:Saving checkpoints for 151416 into ../dfs/checkpoint/customized_model-c/model.ckpt.
INFO:tensorflow:global_step/sec: 1.1454
INFO:tensorflow:loss = 0.00232475, step = 151502 (87.306 sec)
INFO:tensorflow:global_step/sec: 1.15126
INFO:tensorflow:loss = 0.00160023, step = 151602 (86.861 sec)
INFO:tensorflow:global_step/sec: 1.14986
INFO:tensorflow:loss = 0.00205829, step = 151702 (86.968 sec)
INFO:tensorflow:global_step/sec: 1.15003
INFO:tensorflow:loss = 0.00095673, step = 151802 (86.954 sec)
INFO:tensorflow:global_step/sec: 1.15031
INFO:tensorflow:loss = 0.00157477, step = 151902 (86.933 sec)
INFO:tensorflow:global_step/sec: 1.15178
INFO:tensorflow:loss = 0.00251543, step = 152002 (86.822 sec)
INFO:tensorflow:global_step/sec: 1.14925
INFO:tensorflow:loss = 0.00269076, step = 152102 (87.014 sec)
INFO:tensorflow:Saving checkpoints for 152106 into ../dfs/checkpoint/customize

INFO:tensorflow:global_step/sec: 1.10128
INFO:tensorflow:loss = 0.00225495, step = 158302 (90.803 sec)
INFO:tensorflow:global_step/sec: 1.10046
INFO:tensorflow:loss = 0.00270049, step = 158402 (90.871 sec)
INFO:tensorflow:global_step/sec: 1.10049
INFO:tensorflow:loss = 0.0022854, step = 158502 (90.869 sec)
INFO:tensorflow:global_step/sec: 1.09872
INFO:tensorflow:loss = 0.000877302, step = 158602 (91.015 sec)
INFO:tensorflow:global_step/sec: 1.09975
INFO:tensorflow:loss = 0.00469244, step = 158702 (90.930 sec)
INFO:tensorflow:Saving checkpoints for 158757 into ../dfs/checkpoint/customized_model-c/model.ckpt.
INFO:tensorflow:global_step/sec: 1.08221
INFO:tensorflow:loss = 0.00204422, step = 158802 (92.404 sec)
INFO:tensorflow:global_step/sec: 1.09363
INFO:tensorflow:loss = 0.00396761, step = 158902 (91.438 sec)
INFO:tensorflow:global_step/sec: 1.09324
INFO:tensorflow:loss = 0.00134501, step = 159002 (91.471 sec)
INFO:tensorflow:global_step/sec: 1.09233
INFO:tensorflow:loss = 0.000644248,

INFO:tensorflow:global_step/sec: 1.14155
INFO:tensorflow:loss = 0.0089003, step = 165302 (87.600 sec)
INFO:tensorflow:global_step/sec: 1.14131
INFO:tensorflow:loss = 0.00152682, step = 165402 (87.620 sec)
INFO:tensorflow:global_step/sec: 1.13845
INFO:tensorflow:loss = 0.0021712, step = 165502 (87.838 sec)
INFO:tensorflow:Saving checkpoints for 165515 into ../dfs/checkpoint/customized_model-c/model.ckpt.
INFO:tensorflow:global_step/sec: 1.12803
INFO:tensorflow:loss = 0.00706161, step = 165602 (88.651 sec)
INFO:tensorflow:global_step/sec: 1.13748
INFO:tensorflow:loss = 0.0120153, step = 165702 (87.914 sec)
INFO:tensorflow:global_step/sec: 1.13763
INFO:tensorflow:loss = 0.00356306, step = 165802 (87.902 sec)
INFO:tensorflow:global_step/sec: 1.13401
INFO:tensorflow:loss = 0.000918012, step = 165902 (88.183 sec)
INFO:tensorflow:global_step/sec: 1.13743
INFO:tensorflow:loss = 0.00170123, step = 166002 (87.917 sec)
INFO:tensorflow:global_step/sec: 1.13773
INFO:tensorflow:loss = 0.0012735, ste

INFO:tensorflow:loss = 0.000816924, step = 172202 (92.389 sec)
INFO:tensorflow:global_step/sec: 1.08472
INFO:tensorflow:loss = 0.00152861, step = 172302 (92.190 sec)
INFO:tensorflow:global_step/sec: 1.0873
INFO:tensorflow:loss = 0.0015641, step = 172402 (91.970 sec)
INFO:tensorflow:global_step/sec: 1.08699
INFO:tensorflow:loss = 0.000847322, step = 172502 (91.997 sec)
INFO:tensorflow:global_step/sec: 1.08105
INFO:tensorflow:loss = 0.00135515, step = 172602 (92.503 sec)
INFO:tensorflow:global_step/sec: 1.08301
INFO:tensorflow:loss = 0.000404401, step = 172702 (92.335 sec)
INFO:tensorflow:Saving checkpoints for 172750 into ../dfs/checkpoint/customized_model-c/model.ckpt.
INFO:tensorflow:global_step/sec: 1.07773
INFO:tensorflow:loss = 0.00092093, step = 172802 (92.787 sec)
INFO:tensorflow:global_step/sec: 1.09369
INFO:tensorflow:loss = 0.00322767, step = 172902 (91.434 sec)
INFO:tensorflow:global_step/sec: 1.09152
INFO:tensorflow:loss = 0.00831809, step = 173002 (91.616 sec)
INFO:tensorfl

...

INFO:tensorflow:loss = 0.00423059, step = 213302 (144.905 sec)
INFO:tensorflow:Saving checkpoints for 213398 into ../dfs/checkpoint/customized_model-c/model.ckpt.
INFO:tensorflow:global_step/sec: 0.677455
INFO:tensorflow:loss = 0.00428305, step = 213402 (147.605 sec)
INFO:tensorflow:global_step/sec: 0.686625
INFO:tensorflow:loss = 0.00133777, step = 213502 (145.647 sec)
INFO:tensorflow:global_step/sec: 0.687117
INFO:tensorflow:loss = 0.000897009, step = 213602 (145.530 sec)
INFO:tensorflow:global_step/sec: 0.68845
INFO:tensorflow:loss = 0.000320035, step = 213702 (145.254 sec)
INFO:tensorflow:global_step/sec: 0.685296
INFO:tensorflow:loss = 0.00619005, step = 213802 (145.924 sec)
INFO:tensorflow:Saving checkpoints for 213810 into ../dfs/checkpoint/customized_model-c/model.ckpt.
INFO:tensorflow:global_step/sec: 0.678971
INFO:tensorflow:loss = 0.000932574, step = 213902 (147.280 sec)
INFO:tensorflow:global_step/sec: 0.685695
INFO:tensorflow:loss = 0.00226417, step = 214002 (145.838 sec)


INFO:tensorflow:global_step/sec: 1.15683
INFO:tensorflow:loss = 0.00719102, step = 220202 (86.444 sec)
INFO:tensorflow:global_step/sec: 1.15237
INFO:tensorflow:loss = 0.00314578, step = 220302 (86.778 sec)
INFO:tensorflow:global_step/sec: 1.15695
INFO:tensorflow:loss = 0.00110487, step = 220402 (86.434 sec)
INFO:tensorflow:global_step/sec: 1.15647
INFO:tensorflow:loss = 0.000460939, step = 220502 (86.471 sec)
INFO:tensorflow:global_step/sec: 1.15862
INFO:tensorflow:loss = 0.000931468, step = 220602 (86.309 sec)
INFO:tensorflow:global_step/sec: 1.15772
INFO:tensorflow:loss = 0.0010019, step = 220702 (86.376 sec)
INFO:tensorflow:Saving checkpoints for 220766 into ../dfs/checkpoint/customized_model-c/model.ckpt.
INFO:tensorflow:global_step/sec: 1.144
INFO:tensorflow:loss = 0.00122731, step = 220802 (87.413 sec)
INFO:tensorflow:global_step/sec: 1.15658
INFO:tensorflow:loss = 0.00190697, step = 220902 (86.462 sec)
INFO:tensorflow:global_step/sec: 1.15522
INFO:tensorflow:loss = 0.0012453, st

INFO:tensorflow:global_step/sec: 1.15008
INFO:tensorflow:loss = 0.00515454, step = 227302 (86.949 sec)
INFO:tensorflow:global_step/sec: 1.15116
INFO:tensorflow:loss = 0.00149834, step = 227402 (86.869 sec)
INFO:tensorflow:global_step/sec: 1.14987
INFO:tensorflow:loss = 0.00157543, step = 227502 (86.966 sec)
INFO:tensorflow:global_step/sec: 1.15168
INFO:tensorflow:loss = 0.00232341, step = 227602 (86.830 sec)
INFO:tensorflow:Saving checkpoints for 227680 into ../dfs/checkpoint/customized_model-c/model.ckpt.
INFO:tensorflow:global_step/sec: 1.14178
INFO:tensorflow:loss = 0.000420723, step = 227702 (87.583 sec)
INFO:tensorflow:global_step/sec: 1.15489
INFO:tensorflow:loss = 0.000619381, step = 227802 (86.589 sec)
INFO:tensorflow:global_step/sec: 1.15432
INFO:tensorflow:loss = 0.00161252, step = 227902 (86.631 sec)
INFO:tensorflow:global_step/sec: 1.154
INFO:tensorflow:loss = 0.000264349, step = 228002 (86.656 sec)
INFO:tensorflow:global_step/sec: 1.15654
INFO:tensorflow:loss = 0.000864829

INFO:tensorflow:global_step/sec: 1.14688
INFO:tensorflow:loss = 0.00076642, step = 234302 (87.193 sec)
INFO:tensorflow:global_step/sec: 1.15023
INFO:tensorflow:loss = 0.000768643, step = 234402 (86.939 sec)
INFO:tensorflow:global_step/sec: 1.14735
INFO:tensorflow:loss = 0.00214586, step = 234502 (87.159 sec)
INFO:tensorflow:Saving checkpoints for 234597 into ../dfs/checkpoint/customized_model-c/model.ckpt.
INFO:tensorflow:global_step/sec: 1.13847
INFO:tensorflow:loss = 0.00116555, step = 234602 (87.835 sec)
INFO:tensorflow:global_step/sec: 1.15397
INFO:tensorflow:loss = 0.000800297, step = 234702 (86.658 sec)
INFO:tensorflow:global_step/sec: 1.15359
INFO:tensorflow:loss = 0.00204004, step = 234802 (86.686 sec)
INFO:tensorflow:global_step/sec: 1.15184
INFO:tensorflow:loss = 0.00117316, step = 234902 (86.818 sec)
INFO:tensorflow:global_step/sec: 1.14978
INFO:tensorflow:loss = 0.000655961, step = 235002 (86.973 sec)
INFO:tensorflow:global_step/sec: 1.15482
INFO:tensorflow:loss = 0.0013594

INFO:tensorflow:global_step/sec: 1.14481
INFO:tensorflow:loss = 0.001244, step = 241302 (87.351 sec)
INFO:tensorflow:global_step/sec: 1.14554
INFO:tensorflow:loss = 0.000994571, step = 241402 (87.295 sec)
INFO:tensorflow:Saving checkpoints for 241492 into ../dfs/checkpoint/customized_model-c/model.ckpt.
INFO:tensorflow:global_step/sec: 1.13407
INFO:tensorflow:loss = 0.000853965, step = 241502 (88.178 sec)
INFO:tensorflow:global_step/sec: 1.14836
INFO:tensorflow:loss = 0.00250615, step = 241602 (87.081 sec)
INFO:tensorflow:global_step/sec: 1.15244
INFO:tensorflow:loss = 0.0125295, step = 241702 (86.773 sec)
INFO:tensorflow:global_step/sec: 1.14887
INFO:tensorflow:loss = 0.00104157, step = 241802 (87.042 sec)
INFO:tensorflow:global_step/sec: 1.14937
INFO:tensorflow:loss = 0.000600576, step = 241902 (87.004 sec)
INFO:tensorflow:global_step/sec: 1.14985
INFO:tensorflow:loss = 0.00486999, step = 242002 (86.968 sec)
INFO:tensorflow:global_step/sec: 1.14794
INFO:tensorflow:loss = 0.00197199, 

-> 关于kernel size:
TIP: If filter height and width have the same value, you can instead specify a single integer for kernel_size—e.g., kernel_size=5.

tensorflow 官网上MNIST示例，是5*5： https://www.tensorflow.org/versions/master/tutorials/layers

Very small filter sizes will capture very fine details of the image. On the other hand having a bigger filter size will leave out minute details in the image.
如果图片细节重要，则需要kernel size小
https://www.quora.com/How-can-I-decide-the-kernel-size-output-maps-and-layers-of-CNN

这个好像也没有固定答案。一般建议 3*3 , 5*5 或7*7。 一般而言，3*3可能会好些。
https://stats.stackexchange.com/questions/296679/what-does-kernel-size-mean



-> 什么叫 activation function

https://towardsdatascience.com/activation-functions-and-its-types-which-is-better-a9a5310cc8f

http://www.holehouse.org/mlclass/06_Logistic_Regression_files/Image%20[1].png



http://www.holehouse.org/mlclass/
By activation, we mean the value which is computed and output by that node


    Neural networks were developed as a way to simulate networks of neurones
    
    This is an artificial neurone with a sigmoid (logistic) activation function
    关于激活函数(activation function)，其实说来话长。首先，需要知道神经网络的由来
    
    
神经网络，就是对人脑神经元工作的模拟:

![神经元](http://www.holehouse.org/mlclass/08_Neural_Networks_Representation_files/Image%20[2].png)
![神经网络](http://www.holehouse.org/mlclass/08_Neural_Networks_Representation_files/Image%20[4].png)

可以把上面第2个图的圈圈，想像成一个个神经元。

当然，圈圈都是一个个数；那，这些数怎么算呢？第一层，叫输入层；当然，不论哪层的哪个神经元，都应该有用。那怎么叫有用呢？就是激活它，就是通过它，得到下一层神经元（圈圈）的值；如何激活它？就是使用一个函数，把这个被激活的神经元，或圈圈的数，代到这个函数中；当然，这个函数，就被称为激活函数 （activation function）了。

常见的激活函数，就是大名鼎鼎的sigmoid函数
![sigmoid函数](http://www.holehouse.org/mlclass/06_Logistic_Regression_files/Image%20[1].png)


sigmoid 出场
![sigmoid函数图像](http://www.holehouse.org/mlclass/06_Logistic_Regression_files/Image%20[2].png)
这个函数的目的，就是将无论什么输入，都弄到0和1之间。即这个函数的y值，就是在0，1之间。





自定义CNN雏形完成：
INFO:tensorflow:Starting evaluation at 2018-04-26-06:35:15
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-04-26-06:35:21
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.106335, global_step = 1000, loss = 3.58817

Test Accuracy: 10.633484%

INFO:tensorflow:Starting evaluation at 2018-04-26-11:38:19
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-11101
INFO:tensorflow:Finished evaluation at 2018-04-26-11:38:24
INFO:tensorflow:Saving dict for global step 11101: accuracy = 0.78552, global_step = 11101, loss = 0.791613

Test Accuracy: 78.552037%

10000步时，精度不错。

13000时：
INFO:tensorflow:Starting evaluation at 2018-04-26-12:14:59
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-13101
INFO:tensorflow:Finished evaluation at 2018-04-26-12:15:04
INFO:tensorflow:Saving dict for global step 13101: accuracy = 0.80543, global_step = 13101, loss = 0.680529

Test Accuracy: 80.542988%

INFO:tensorflow:Starting evaluation at 2018-04-26-13:40:56
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-15101
INFO:tensorflow:Finished evaluation at 2018-04-26-13:41:02
INFO:tensorflow:Saving dict for global step 15101: accuracy = 0.826697, global_step = 15101, loss = 0.62195

Test Accuracy: 82.669681%
 
 INFO:tensorflow:Starting evaluation at 2018-04-27-10:11:28
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-95101
INFO:tensorflow:Finished evaluation at 2018-04-27-10:11:35
INFO:tensorflow:Saving dict for global step 95101: accuracy = 0.893665, global_step = 95101, loss = 0.469633

Test Accuracy: 89.366513%

INFO:tensorflow:Starting evaluation at 2018-04-29-02:00:37
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-115101
INFO:tensorflow:Finished evaluation at 2018-04-29-02:00:43
INFO:tensorflow:Saving dict for global step 115101: accuracy = 0.893665, global_step = 115101, loss = 0.471583

Test Accuracy: 89.366513%

下面仅做测试：

INFO:tensorflow:Starting evaluation at 2018-04-29-04:28:24
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-125101
INFO:tensorflow:Finished evaluation at 2018-04-29-04:28:30
INFO:tensorflow:Saving dict for global step 125101: accuracy = 0.895475, global_step = 125101, loss = 0.492467

Test Accuracy: 89.547509%

INFO:tensorflow:Starting evaluation at 2018-04-29-13:00:24
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-145101
INFO:tensorflow:Finished evaluation at 2018-04-29-13:00:30
INFO:tensorflow:Saving dict for global step 145101: accuracy = 0.893665, global_step = 145101, loss = 0.504544

Test Accuracy: 89.366513%

INFO:tensorflow:Starting evaluation at 2018-05-01-01:01:11
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-245101
INFO:tensorflow:Finished evaluation at 2018-05-01-01:01:17
INFO:tensorflow:Saving dict for global step 245101: accuracy = 0.897738, global_step = 245101, loss = 0.524077

Test Accuracy: 89.773756%

这个用了10万步，只前进了0.4%，看来，应该是难有进步了。想达到97%，是不大可能了。
解决的方法，可能是应用gamma随机校正，来增加训练数据的复杂性。可能会提高准确度。

In [2]:
# from: https://www.kaggle.com/jeffcarp/example-save-and-load-a-tensorflow-model

import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

from tensorflow.contrib.learn.python.learn.estimators import run_config
from tensorflow.contrib.training.python.training import hparam

IMAGE_SIZE = 64
# CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 64, 64, 1])
  print ('shape of input_layer: ', input_layer)
  # with batch_size =100, shape should be: [100, 28, 28, 1]
  # shape of input_layer:  Tensor("Reshape:0", shape=(100, 64, 64, 1), dtype=float32)

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  print ('shape of conv1: ', conv1)
  # shape of conv1:  Tensor("conv2d/Relu:0", shape=(100, 64, 64, 32), dtype=float32)
    
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  print ('shape of conv2: ', conv2, '; and shape of pool2 is: ', pool2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=CHARSET_SIZE)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

class DataSetLoader:
    def __init__(self, data_dir):
        # Set CHARSET_SIZE to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        self.images = np.float32(self.images)
        # or else: TypeError: Value passed to parameter 'input' has DataType float64 not in list of allowed values: float16, float32

        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]

# Build CNN with customized function ...
classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn,
   model_dir='../dfs/checkpoint/customized_model-c')



# MODEL_DIR = "../dfs/checkpoint/customized_model-c"
# model_from_checkpoint = make_estimator(MODEL_DIR)

test_data = DataSetLoader(data_dir='../data/test_/')
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../dfs/checkpoint/customized_model-c', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
Now processing path:  ../data/test_/00037
shape of input_layer:  Tensor("Reshape:0", shape=(?, 64, 64, 1), dtype=float32)
shape of conv1:  Tensor("conv2d/Relu:0", shape=(?, 64, 64, 32), dtype=float32)
shape of conv2:  Tensor("conv2d_2/Relu:0", shape=(?, 32, 32, 64), dtype=float32) ; and shape of pool2 is:  Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
INFO:tensorflow:Starting evaluation at 2018-05-01-01:01:11
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/customized_model-c/model.ckpt-245101
INFO:tensorflow:Finished evaluation at 2018-05-01-01:01:17
INFO:tensorflow:Saving dict for global step